In [2]:
# mnist_cnn_deep
# MNIST and Convolutional Neural Network
# L1,L2 : conv2d + relu + max_pool 
# L3 : FC(Fully Connected Layer)

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
tf.random.set_seed(5)

In [5]:
# mnist 데이터 가져오기
mnist = tf.keras.datasets.mnist
(x_train,y_train),(x_test,y_test) = mnist.load_data()
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [6]:
# one-hot 인코딩
nb_classes = 10   # 분류 class의 갯수(0~9)

Y_one_hot = tf.one_hot(y_train,nb_classes) # (60000,)
Y_one_hot.shape                            # (60000,10)  # 2차원

TensorShape([60000, 10])

In [10]:
# X값의 타입을 float형으로 변환
x_train = tf.cast(x_train,dtype=tf.float32)
x_test = tf.cast(x_test,dtype=tf.float32)

In [14]:
# X값의 shape을 4차원으로 변환
X_img = tf.reshape(x_train,[-1,28,28,1])
print(X_img.shape)  # (60000, 28, 28, 1)

(60000, 28, 28, 1)
